## Visualize collected `geospatial data` on a <span style="color:turquoise;">MAP</span>

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() 
Bearer = os.getenv('Bearer')
apiurl = os.getenv('apiurl')


In [2]:
import requests
from pprint import pprint
#Define the bldg code service function
#ONU
#Enter user number
num = int(input("Please enter a number: "))
def getONU(num):
    url = f"{apiurl}/api/komp_assistant/onu/phone/{num}"

    payload = {}
    headers = {
    'Authorization': Bearer
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        #pprint(data)
        bcode_value = data.get("bcode")
        bname_value = data.get("bname")
        return bcode_value, bname_value
    else:
        bcode_value = None
        return None

def getPAP(num):
    url = f"{apiurl}/api/komp_assistant/phone/{num}"

    payload = {}
    headers = {
    'Authorization': Bearer
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        bcode_value = data.get("bcode")
        bname_value = data.get("bname")
        return bcode_value, bname_value
    else:
        bcode_value = None
        return None

def getbcode(num):
    try:
        bcode, bname = getPAP(num)
        if bcode == None:
            bcode, bname = getONU(num)
            if bcode == None:
                print('Bcode not found for both PAP and ONU')
                return 'Not Found'
            else:
                #print('ONU', num, bcode)
                return 'ONU', num, bcode, bname
        else:
            #print('PAP', num, bcode)
            return 'PAP', num, bcode, bname
    except Exception as e:
        print('Error: ',e)
result = getbcode(num)
print(result)
if isinstance(result, tuple):
    print("Result is a tuple")
    router, num, buildingcode, bname = result
else:
    print("Result is not a tuple")
    buildingcode = 'Not Exist'


('PAP', 706139071, 'KENRZ18020', 'Harvest')
Result is a tuple


In [3]:
#%pip install folium
import zipfile
import xml.etree.ElementTree as ET
import folium

with zipfile.ZipFile('./geobank/KCIS Master Map 3010_2023.kmz', 'r') as zip_ref:
    zip_ref.extractall('./geobank/')

# Parse the .kml file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

In [21]:
for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
    #print(name)

In [22]:
print(buildingcode)
buildingcode = str(buildingcode)
for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
    #print(name)
    if buildingcode in name: 
        coordinates = placemark.find('{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates').text
        # coordinates will be a string in the format "longitude,latitude,altitude"
        # you can split it into a list of floats using:
        coordinates_list = [float(coord) for coord in coordinates.split(',')]
        print(coordinates_list)


KENKS07260
[36.9171650676369, -1.236455735884236, 0.0]


In [23]:
longitude, latitude = coordinates_list[:2]
# Create a folium map centered around the coordinates
map_object = folium.Map(location=[latitude, longitude], zoom_start=15)

# Add a marker to the map
folium.Marker(location=[latitude, longitude], popup=f'Contact: {num}, Router Type: {router}, Building: {bname}').add_to(map_object)

# Save the map to an HTML file
map_object.save('./geobank/map.html')


In [4]:
import sqlite3
from pprint import pprint

# Connect to the SQLite database
conn = sqlite3.connect('transcriptions.db')
cursor = conn.cursor()

# Initialize an empty dictionary to store results
results_dict = {}

# Retrieve unique contact numbers from the transcriptions table
cursor.execute("SELECT DISTINCT contact FROM transcriptions WHERE contact IS NOT NULL;")
# cursor.execute("SELECT DISTINCT contact FROM transcriptions WHERE contact IS NOT NULL AND (substr(date, 1, 10) = '2023-10-28' AND issue_category = 'Router Technical Problem');")


contacts = cursor.fetchall()

# Iterate over each contact number
for contact in contacts:
    num = contact[0]

    # Use the getbcode function to get router, num, building code, and bname
    result = getbcode(num)

    if isinstance(result, tuple):
        router, num, buildingcode, bname = result
    else:
        buildingcode = 'Not Exist'

    # Store the results in the dictionary
    results_dict[num] = {
        'router': router,
        'num': num,
        'buildingcode': buildingcode,
        'bname': bname
    }

# Close the database connection
conn.close()

# Print the results dictionary
pprint(results_dict)


Bcode not found for both PAP and ONU
Bcode not found for both PAP and ONU
{'114049713': {'bname': 'Wanganga',
               'buildingcode': 'KENRZ16108',
               'num': '114049713',
               'router': 'PAP'},
 '701263734': {'bname': 'Amanda',
               'buildingcode': 'KENKH07249',
               'num': '701263734',
               'router': 'ONU'},
 '703747241': {'bname': 'Cole',
               'buildingcode': 'KENKM07061',
               'num': '703747241',
               'router': 'ONU'},
 '704423297': {'bname': 'Elimar',
               'buildingcode': 'KENRZ10132',
               'num': '704423297',
               'router': 'PAP'},
 '704668899': {'bname': 'Balabala Hse',
               'buildingcode': 'KENRR13034',
               'num': '704668899',
               'router': 'PAP'},
 '705173447': {'bname': 'Anaya Hse',
               'buildingcode': 'KENKH28022',
               'num': '705173447',
               'router': 'PAP'},
 '705899247': {'bname': 'Spring boa

In [25]:
import folium
from regex import B

# Initialize an empty folium map
map_object = folium.Map(location=[-1.2921, 36.8219], zoom_start=10)  # Default centre initialized Nairobi, Kenya and zoom

# Iterate over each user in the results_dict
for num, user_info in results_dict.items():
    # Extract user information
    router = user_info['router']
    buildingcode = user_info['buildingcode']
    if 'KENKH' in buildingcode:
        iconcolor = 'blue'
    elif 'KENKS' in buildingcode:
        iconcolor = 'darkblue'
    elif 'KENKM' in buildingcode:
        iconcolor = 'purple'
    elif 'KEKRK' in buildingcode:
        iconcolor = 'cadetblue'
    elif 'KENKK' in buildingcode:
        iconcolor = 'gray'
    elif 'KENRG' in buildingcode:
        iconcolor = 'black'
    elif 'KENRZ' in buildingcode:
        iconcolor = 'beige'
    elif 'KEKRG' in buildingcode:
        iconcolor = 'lightblue'
    elif 'KENRK' in buildingcode:
        iconcolor = 'green'
    elif 'KENRL' in buildingcode:
        iconcolor = 'lightgray'
    elif 'KENRR' in buildingcode:
        iconcolor = 'darkblue'
    else:
        iconcolor = 'orange'

    bname = user_info['bname']

    # Check if building code is available
    if buildingcode and ('Not Exist' not in buildingcode) and (isinstance(buildingcode, str) or isinstance(buildingcode, tuple)):
        if isinstance(buildingcode, str):
            buildingcode_list = [buildingcode]
        else:
            buildingcode_list = list(buildingcode)

        for bcode in buildingcode_list:
            # Iterate over each placemark in the KML file
            for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
                name = placemark.find('{http://www.opengis.net/kml/2.2}name').text

                # Check if the building code is in the name
                if bcode in name:
                    coordinates = placemark.find('{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates').text
                    coordinates_list = [float(coord) for coord in coordinates.split(',')]

                    # Extract latitude and longitude
                    longitude, latitude = coordinates_list[:2]

                    # Add a marker for each user to the folium map
                    folium.Marker(
                        location=[latitude, longitude],
                        popup=f'Contact: {num}, Router Type: {router}, Building: {bname}',icon=folium.Icon(color=iconcolor, icon='earphone' )
                    ).add_to(map_object)

# Save the map to an HTML file
map_object.save('./geobank/mapwithoutoutlines.html')


In [1]:
import folium
from xml.etree import ElementTree as ET

def add_polygon(map_object, coordinates, name):
    # Extract latitudes, longitudes, and altitudes from the coordinates
    latitudes = [coord[1] for coord in coordinates]
    longitudes = [coord[0] for coord in coordinates]

    # Check if there are coordinates before calculating the center
    if latitudes and longitudes:
        # Find the center of the polygon
        center_lat = sum(latitudes) / len(latitudes)
        center_lon = sum(longitudes) / len(longitudes)

        # Create a Folium polygon
        folium.Polygon(
            locations=[(lon, lat) for lon, lat, _ in coordinates],  # Use all three values (lat, lon, _)
            color='blue',
            fill=True,
            fill_color='lightblue',
            fill_opacity=0.4,
            popup=name,
            location=(center_lat, center_lon)  # Set the center of the polygon
        ).add_to(map_object)

def process_folder(folder, map_object):
    for placemark in folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
        coordinates_text = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
        coordinates = [list(map(float, coord.split(','))) for coord in coordinates_text.split()]
        add_polygon(map_object, coordinates, name)

# Parse the KML file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

# Create a Folium map
map_object = folium.Map(location=[0, 0], zoom_start=3)

# Process top-level folders
for folder in root.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
    process_folder(folder, map_object)

# Save or display the map
map_object.save('./geobank/map_with_outlines.html')


### As observed the created html file is too large to render on browser without being `CPU intensive`. 
#### Solution: 
Externalize Data: Instead of embedding all data directly into the HTML file, consider loading data dynamically from an external source (e.g., GeoJSON files) as needed. This can improve initial loading times.

In [15]:
#%pip install geopandas
#%pip install Fiona pyproj
# %pip install osgeo
# %pip install GDAL
#%pip install geopy
import geopandas as gpd
from geopandas.tools import geocode
import fiona

# Enable KML driver
fiona.drvsupport.supported_drivers['KML'] = 'rw'

# Load KML file using geopandas.tools
kml_file_path = './geobank/doc.kml'
gdf = gpd.read_file(kml_file_path, provider='ogr')

# Save GeoJSON file
geojson_file_path = './geobank/output.geojson'
gdf.to_file(geojson_file_path, driver='GeoJSON')


In [19]:
import folium
import json

# Load GeoJSON data from the file
with open('./geobank/output.geojson', 'r') as f:
    geojson_data = json.load(f)

# Calculate the average latitude and longitude for the center
all_coordinates = []

for feature in geojson_data['features']:
    geometry_type = feature['geometry']['type']

    if geometry_type == 'Point':
        all_coordinates.append(feature['geometry']['coordinates'])
    elif geometry_type == 'MultiPoint':
        all_coordinates.extend(feature['geometry']['coordinates'])

avg_latitude = sum(coord[1] for coord in all_coordinates) / len(all_coordinates)
avg_longitude = sum(coord[0] for coord in all_coordinates) / len(all_coordinates)

# Create a Folium Map centered at the average latitude and longitude
map_center = [avg_latitude, avg_longitude]
mymap = folium.Map(location=map_center, zoom_start=12, control_scale=True)

# Add GeoJSON layer to the map
folium.GeoJson(geojson_data).add_to(mymap)

# Save the map as an HTML file
mymap.save("./geobank/geojsonmap.html")





In [32]:
import geopandas as gpd
import pandas as pd

def process_folder(folder, gdf_list):
    for placemark in folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
        coordinates_text = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
        coordinates = [list(map(float, coord.split(','))) for coord in coordinates_text.split()]

        # Check if all coordinates have the same length
        if len(set(len(coord) for coord in coordinates)) != 1:
            print(f"Skipping placemark '{name}' due to varying lengths of coordinates.")
            continue

        # Create a temporary GeoDataFrame with the placemark data
        gdf_temp = gpd.GeoDataFrame({'Name': [name], 'geometry': gpd.points_from_xy(*zip(*coordinates))})

        # Append it to the list of GeoDataFrames
        gdf_list.append(gdf_temp)

    for subfolder in folder.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
        gdf_list = process_folder(subfolder, gdf_list)

    return gdf_list

# Parse the KML file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

# Create an empty list to store GeoDataFrames
gdf_list = []

# Process top-level folders
for folder in root.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
    gdf_list = process_folder(folder, gdf_list)

# Concatenate the list of GeoDataFrames into one GeoDataFrame
gdf = pd.concat(gdf_list, ignore_index=True)

# Save GeoJSON file
gdf.to_file('./geobank/output2.geojson', driver='GeoJSON')


## Brotli Compression

In [1]:
%pip install brotli
import brotli

with open('./geobank/map_with_outlines.html', 'rb') as input_file:
    compressed_data = brotli.compress(input_file.read())

with open('./geobank/map_with_outlines.html.br', 'wb') as output_file:
    output_file.write(compressed_data)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.7 kB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [4]:
import brotli

with open('./geobank/map_with_outlines.html.br', 'rb') as input_file:
    compressed_data = input_file.read()

decompressed_data = brotli.decompress(compressed_data)

with open('./geobank/decompressed_file.html', 'wb') as output_file:
    output_file.write(decompressed_data)

### Simplifying Polygons with `Folium.PolyLine` method's `Smooth Factor`

In [6]:
import folium
from xml.etree import ElementTree as ET

def add_polygon(map_object, coordinates, name):
    # Extract latitudes, longitudes, and altitudes from the coordinates
    longitudes = [coord[0] for coord in coordinates]
    latitudes = [coord[1] for coord in coordinates]

    # Check if there are coordinates before calculating the center
    if latitudes and longitudes:
        # Find the center of the polygon
        center_lat = sum(latitudes) / len(latitudes)
        center_lon = sum(longitudes) / len(longitudes)

        # Create a Folium polyline with a smooth factor
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat, _ in coordinates],  # Adjust the order here
            color='blue',
            weight=2,
            smooth_factor=0.5,  # Adjust the smooth factor (0.0 to 1.0)
            popup=name,
        ).add_to(map_object)
def process_folder(folder, map_object):
    for placemark in folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
        coordinates_text = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
        coordinates = [list(map(float, coord.split(','))) for coord in coordinates_text.split()]
        add_polygon(map_object, coordinates, name)

# Parse the KML file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

# Create a Folium map
map_object = folium.Map(location=[0, 0], zoom_start=3)

# Process top-level folders
for folder in root.findall('.//{http://www.opengis.net/kml/2.2}Folder'):
    process_folder(folder, map_object)

# Save or display the map
map_object.save('./geobank/map_with_smoothoutlines.html')